In [1]:
import cv2                   
from deepface import DeepFace  
import numpy as np             

def main():               
    vid = cv2.VideoCapture(0)  
    if not vid.isOpened():     
        return            

    last_region = None # guarda la última posición del rostro detectado 
    font = cv2.FONT_HERSHEY_SIMPLEX  # font

    try:                        
        while True:            
            ret, frame = vid.read()  
            if not ret: break 

            frame = cv2.flip(frame, 1) # invierte la imagen horizontalmente

            current_region = None # inicializa la región del rostro para este frame como None
            try:
                # Detección del rostro
                faces = DeepFace.extract_faces( # usa DeepFace para detectar rostros en el frame
                    img_path=frame, # pasa directamente el frame (imagen) en lugar de una ruta
                    enforce_detection=False, # no lanza error si no encuentra rostros (devuelve una lista vacía)
                    detector_backend='opencv' # usa el backend de detección facial de OpenCV
                )
                if len(faces) > 0: 
                    current_region = faces[0] ['facial_area'] # toma el primer rostro detectado # ['facial_area']  es un diccionario con las coordenadas y dimensiones del rectángulo del rostro
                    last_region = current_region # actualiza también last_region con esta nueva posición
            except Exception:
                pass # si hay un error en la detección, lo ignora y sigue sin que el programa se caiga

            # si en el frame actual tenemos un rostro, usamos ese, si no, usamos el último rostro encontrado
            region = current_region if current_region is not None else last_region

            if region is not None: # si tenemos una región válida (rostro detectado ahora o en un frame anterior)
                x = region['x'] # coordenada x (horizontal) de la esquina superior izquierda del rostro
                y = region['y'] # coordenada y (vertical) de la esquina superior izquierda del rostro
                w = region['w'] # anchura del rectángulo del rostro
                h = region['h'] # altura del rectángulo del rostro
                
                # censura sobre ojos
                # los ojos están aproximadamente entre el 20% y el 50% de la altura del rostro desde arriba
                eye_y_start = y + int(h * 0.25) # posición vertical de inicio de la barra 
                eye_h = int(h * 0.2) # altura de la barra negra
                

                cv2.rectangle(
                    frame, # imagen sobre la que se dibuja
                    (x, eye_y_start), # punto superior izquierdo del rectángulo
                    (x + w, eye_y_start + eye_h),# punto inferior derecho del rectángulo
                    (0, 0, 0), # color negro (B, G, R)
                    -1 # grosor -1 = rectángulo relleno
                )
                
                # texto "CENSORED"
                text = "CENSORED"

                # calcula el tamaño del texto para una escala inicial 
                (text_w, text_h), _ = cv2.getTextSize(text, font, 0.8, 2)

                # ajusta la escala de la fuente de forma proporcional al ancho del rostro
                font_scale = 0.8 * (w / 300.0) # escala de la fuente proporcional a la anchura del rostro
                if font_scale < 0.3: # impone un tamaño mínimo para que sea legible
                    font_scale = 0.3

                # recalcula el tamaño del texto con la escala definitiva
                (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, 2)

                # coord x para centrar horizontalmente el texto dentro del ancho del rostro
                text_x = x + (w - text_w) // 2

                # coord y para centrar verticalmente el texto dentro de la barra de los ojos
                text_y = eye_y_start + (eye_h + text_h) // 2

                # dibuja el texto "CENSORED" blanco encima de la barra negra
                cv2.putText(
                    frame, # Imagen
                    text, # cadena a dibujar
                    (int(text_x), int(text_y)),# posición del texto 
                    font, # fuente elegida
                    font_scale, # escala de la fuente
                    (255, 255, 255), # blanco
                    2, # grosor del trazo
                    cv2.LINE_AA # tipo de línea (antialiasing y texto más suave)
                )

                # cartel "PERSONA BUSCADA"
                sign_w = int(w * 1.2) # anchura del cartel (20% más ancho que el rostro)
                sign_h = int(h * 0.4) # altura del cartel (40% de la altura del rostro)

                # centra el cartel respecto al rostro desplazándolo un poco hacia la izquierda
                sign_x = x - int((sign_w - w) / 2)

                # coloca el cartel debajo de la barbilla 
                sign_y = y + h + 20

                # dibuja el cartel: rectángulo blanco relleno
                cv2.rectangle(
                    frame,
                    (sign_x, sign_y), # esquina superior izquierda del cartel
                    (sign_x + sign_w, sign_y + sign_h), # esquina inferior derecha del cartel
                    (255, 255, 255), # blanco
                    -1 
                )

                # dibuja el borde del cartel: rectángulo negro con grosor 3
                cv2.rectangle(
                    frame,
                    (sign_x, sign_y),
                    (sign_x + sign_w, sign_y + sign_h),
                    (0, 0, 0),# negro
                    3 # grosor del borde
                )

                wanted_text = "PERSONA BUSCADA" 

                # primera estimación del tamaño del texto con escala 1
                (w_text_w, w_text_h), _ = cv2.getTextSize(wanted_text, font, 1, 4)

                # calcula una escala para que el texto quepa horizontalmente en el cartel 
                w_font_scale = 1.0 * (sign_w / (w_text_w + 20))

                # cecalcula el tamaño del texto con la escala adaptada
                (w_text_w, w_text_h), _ = cv2.getTextSize(wanted_text, font, w_font_scale, 4)

                # coord x para centrar el texto horizontalmente en el cartel
                w_text_x = sign_x + (sign_w - w_text_w) // 2

                # coord y para centrar el texto verticalmente en el cartel
                w_text_y = sign_y + (sign_h + w_text_h) // 2

                # escribe en rojo "PERSONA BUSCADA" en el centro del cartel
                cv2.putText(
                    frame,
                    wanted_text,
                    (int(w_text_x), int(w_text_y)), # posición del texto
                    font,
                    w_font_scale, # escala adaptada al cartel
                    (0, 0, 255), # rojo
                    3, # grosor del trazo
                    cv2.LINE_AA # atialiasing
                )

            # muestra el frame (con censura y cartel) en una ventana
            cv2.imshow("CENSORED MODE", frame)

  
            if cv2.waitKey(1) & 0xFF == 27:
                break     
    finally:                  
        vid.release()         
        cv2.destroyAllWindows()
        cv2.waitKey(1)       

if __name__ == "__main__":     
    main()                  
